# Simple Autoencoder

In this notebook we will create a simple autoencoder that just learns a simple system represented by a linear function.

We will use a simple dense network with Stochastic Gradient Descent optimizer. 

## Importing the Libraries.

We begin by importing our libraries:

In [1]:
#Simple Autoencoder

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from sklearn.metrics import mean_squared_error
import numpy as np

# For generating our fake data
import random

We now set some hyperparameters, including # of epochs to train, # of datapoints, as well as noise and # of testing data.

Note that we have DATA_NOISE set at approximately 10% of our maximum and minimum data values, and TEST_NOISE set at 30%.

In [2]:
# Number of epochs to train
num_epochs = 200

# Number of fake datapoints to create
                                                                                                               
DATA_SIZE = 10000
DATA_NOISE = 0.1
VAL_SIZE = int(0.2 * DATA_SIZE)

# Test Noise level
TEST_NOISE = 0.3

## Creating our Simple Autoencoder

We will now create our autoencoder. We use a fully connected dense network with 3 hidden layers with 32, 16 and 32 hidden nodes. We use ReLu activation in all hidden layers and tanh for the output layer, which allows us to create and learn data from \[-1, 1\]

In [3]:
class Autoencoder(nn.Module):
   def __init__(self):
        super(Autoencoder, self).__init__()
        # First hidden layer
        self.l1 = nn.Linear(2, 32)
        self.l2 = nn.Linear(32, 16)
        self.l3 = nn.Linear(16, 32)
        self.l4 = nn.Linear(32, 2)
   def forward(self, x):
        x = self.l1(x)
        x = F.relu(x)
        x = self.l2(x)
        x = F.relu(x)
        x = self.l3(x)
        x = F.relu(x)
        x = self.l4(x)
        output = F.tanh(x)
        return output


## Generating the Data

Let's now generate the data! We set a fixed random seed (42), then create our random datapoints for X between 0 and 1. We then produce our Y data which goes from -1 to 1, and add some synthetic noise.

Finally we call np.array to turn everything into NumPy arrays, which is needed by Keras.

In [4]:
# We set a fixed seed so that our experiments are reproducible
random.seed(42)

# This adds white noise of -scale to scale
def noise(scale):
    return (2 * random.uniform(0, 1) - 1) * scale 

def gen_X(data_size):
    return [random.uniform(0, 1) for i in range(data_size)]

def gen_Y(X):
    return [[x, (2*x + noise(DATA_NOISE) - 1)] for x in X]
# We create our dummy dataset and output, with 1000 numbers
X_in = gen_X(DATA_SIZE)
X_test = gen_X(VAL_SIZE)

# We create a simple linear function with noise for the network to learn
Y = gen_Y(X_in)
Y_out =  torch.tensor(Y)
Y_test =  torch.tensor(gen_Y(X_test))

## Creating Noisy and Wrong Data

We now create our very noisy data, as well as data that is produced using the wrong function ($1.5 \times x$ instead of $2 \times x$), so that we can evaluate how our simple autoencoder behaves.

In [5]:
# We create a noisy version of Y_out to check the anomaly detection
Y_noisy = torch.tensor([[x, 2 * x + noise(TEST_NOISE) - 1] for x in X_in])

# We create an incorrect version of Y_out
Y_wrong = torch.tensor([[x, 1.5*x + noise(DATA_NOISE) - 1]for x in X_in])


## Training and Evaluation

Finally let's train our autoencoder:

In [6]:
model = Autoencoder()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()

loader = data.DataLoader(data.TensorDataset(Y_out, Y_out), batch_size = 1)

def train_model(model):
    for epoch in range(num_epochs + 1):
        model.train()
        for dataX, dataY in loader:
            y = model(dataX)
            loss = loss_fn(y, dataY)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # Validation
        if epoch % 50 != 0:
            continue
        model.eval()
        with torch.no_grad():
            y_pred = model(Y_out)
            train_rmse = np.sqrt(loss_fn(y_pred, Y_out))
            y_pred = model(Y_test)
            test_rmse = np.sqrt(loss_fn(y_pred, Y_test))
        print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))

train_model(model)


Epoch 0: train RMSE 0.0130, test RMSE 0.0128
Epoch 50: train RMSE 0.0063, test RMSE 0.0062
Epoch 100: train RMSE 0.0053, test RMSE 0.0052
Epoch 150: train RMSE 0.0051, test RMSE 0.0049
Epoch 200: train RMSE 0.0060, test RMSE 0.0059


Now let's evaluate the losses for the:

    - Training data
    - Test data
    - Very noisy test data
    - Incorrect data
    
From here we can see how the autoencoder behaves for each type of test:

In [7]:
def test_model(model, X, Y):
    model.eval()
    with torch.no_grad():
        y_pred = model(X)
        test_rmse = np.sqrt(loss_fn(y_pred, Y))
        return test_rmse


clean_loss = test_model(model, Y_out, Y_out)
test_loss = test_model(model, Y_test, Y_test)
noisy_loss = test_model(model, Y_noisy, Y_noisy)
wrong_loss = test_model(model, Y_wrong, Y_wrong)

print("\n\nClean loss = %3.4f, Test loss = %3.4f Noisy loss = %3.4f, Wrong loss = %3.4f" % 
(clean_loss, test_loss, noisy_loss, wrong_loss))




Clean loss = 0.0060, Test loss = 0.0059 Noisy loss = 0.0450, Wrong loss = 0.0681


## Analysis

From the results above, we can see that the test loss is at 0.0059, while the noisy loss and wrong loss are a whole order of magnitude larger at 0.0450 and 0.0681. This allows us to set a threshold of perhaps 0.02 to determine when readings from a system (simulated here by a simple linear system) deviates too far from its normal operation.